<a href="https://colab.research.google.com/github/rkrisanoff/geographical-information-retrieval/blob/main/GIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install natasha
!pip install yandex-geocoder

In [113]:
from pprint import pprint

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

In [ ]:
import glob
from google.colab import drive
drive.mount('/content/drive')
news = glob.glob(f"/content/drive/MyDrive/GIR/FONTANKA-NEWS/news/*.txt")[:15]

In [116]:
# import gdown
# Реальный датасет будет намного больше
# shared_link = 'https://drive.google.com/drive/folders/1YM3wLKR9qJg1SuTTk9UG2iISEObSNtw0?usp=share_link'
# output = 'news'
# news = gdown.download_folder(url=shared_link,output=output,remaining_ok=True,quiet=True) 

In [117]:
documents = dict()

for new_path in news:
  text = ""
  with open(new_path) as new:
    category,title,text = new.readlines()[:3]
    if title[-1:] == '\n':
      title = title[:-1]
  doc = Doc(text)
  doc.segment(segmenter) 
  doc.tag_morph(morph_tagger)
  for token in doc.tokens:
    token.lemmatize(morph_vocab)
  doc.tag_ner(ner_tagger)
  for span in doc.spans:
    span.normalize(morph_vocab)
    span.extract_fact(names_extractor)
    

  documents[title] = []
  for span in doc.spans:
    if span.type == "LOC":
      documents[title].append(span.normal)

In [118]:
for title,words in list(documents.items()):
  print("")
  print(title)
  print('!'+'-'*(len(title)-2)+'!')
  print(words)
  print('^'+'-'*(len(title)-2)+'^')


Сотрудники УБОП закрыли подпольный водочный цех
!---------------------------------------------!
['Мурино', 'Всеволожский район']
^---------------------------------------------^

Губернатор Петербурга и замминистра транспорта летали над КАДом на вертолете
!--------------------------------------------------------------------------!
['Санкт-Петербург', 'РФ', 'Санкт-Петербург', 'РФ', 'Санкт-Петербург', 'Нева', 'Сосновское', 'Мурино', 'Муринский']
^--------------------------------------------------------------------------^

ОЖД ремонтирует платформы и вокзалы
!---------------------------------!
['Кушелевка', 'Пискаревка', 'Финляндское', 'Санкт-Петербург', 'Финляндская']
^---------------------------------^

Ленобласть стала акционером "Зенита"
!----------------------------------!
['Ленинградская область', 'Ленинградская область', 'Мурино', 'Ленинградская область']
^----------------------------------^

«ЗЕНИТ» БУДЕТ БОРОТЬСЯ ЗА МЕСТО В ЗОНЕ УЕФА
!-----------------------------------------!
['

Кластеризация (пока только начало)

In [124]:
from decimal import Decimal
from yandex_geocoder import Client
# insert your api key
client = Client("")

In [ ]:
texts = list(documents.values())
locations = dict()
undefined_locations = set()
for text in texts:
  for location in text:
    if location in undefined_locations:
      continue
    if location not in locations:
      try:
        coord = client.coordinates(location)
        locations[location] = float(coord[0]),float(coord[1])
      except Exception as e:
        print(e)
        undefined_locations.add(location)

In [130]:
undefined_locations

{'Выборгскому', 'Кузьмолово-Мурино', 'Муриной', 'Новодевяткино', 'Финляндское'}

In [129]:
locations

{'Мурино': (30.438578, 60.051284),
 'Всеволожский район': (30.676991, 60.21778),
 'Санкт-Петербург': (30.315644, 59.938955),
 'РФ': (99.505405, 61.698657),
 'Нева': (33.933896, 57.92839),
 'Сосновское': (43.168307, 55.804369),
 'Муринский': (30.398369, 60.027026),
 'Кушелевка': (30.363811, 59.992042),
 'Пискаревка': (30.418007, 59.99062),
 'Финляндская': (25.933922, 62.72321),
 'Ленинградская область': (29.608975, 59.337017),
 'Раменский': (38.208511, 55.478118),
 'Ленобласть': (29.608975, 59.337017),
 'Чехия': (15.267847, 49.574824),
 'Краснодар': (38.975313, 45.03547),
 'Россия': (99.505405, 61.698657),
 'Московский и Всея Русь': (37.365604, 55.538229),
 'Петербург': (30.315644, 59.938955),
 'Валаам': (30.944258, 61.390598),
 'Ладожское озеро': (31.459738, 60.842937),
 'Республика Карелия': (33.232608, 63.621328),
 'Карелия': (33.232608, 63.621328),
 'Валаамская': (37.573097, 55.835609),
 'Валаамский монастырь': (40.007603, 43.592028),
 'Северо-Западный федеральный округ': (36.498137